# Introduction to Swarm 🐝
OpenAI has recently released Swarm, an open-source AI agent orchestration tool designed to streamline the process of building AI agents using clean and simple Python code. With Swarm, you can connect multiple agents to perform complex tasks. This is an exciting tool for those exploring the future of AI automation.

In this notebook, I'll provide an in-depth walkthrough of the Swarm Python library, including how to use it to create an customer support agent system.

## Setting Up Swarm
Before using Swarm, ensure you have Python 3.10 or later installed. To install Swarm, use the following command: `pip install git+https://github.com/openai/swarm.git`

After installation, import the necessary modules to get started:

In [2]:
from swarm import Swarm, Agent
from dotenv import load_dotenv
load_dotenv()

True

Swarm agents use GPT-4o-mini as the default model. To reduce costs, you can explicitly set whatever model you want as your model:

In [3]:
model = 'gpt-4o-mini'

First, we create an entry-point agent (Agent A) that initialises communication and we define a function to transfer control to another agent (Agent B):

In [53]:
def transfer_to_agent_b():
    return agent_b

Then, we create Agent A and define its behavior. In this case, this agent is instructed to be a generic helpful agent, and its function is to transfer the tasks to Agent B.

In [54]:
agent_a = Agent(
    name="Agent A",
    model=model,
    instructions="You are a helpful agent",
    functions=[transfer_to_agent_b]
)

We now create Agent B and define its behavior. In this case, Agent B is going to always give the same response. I have set up this in this way so we can check that it is actually working!

In [55]:
agent_b = Agent(
    name="Agent B",
    model=model,
    instructions="Always reply with 'Bzzzz'",
)

Now, when a user interacts with an agent, OpenAI’s API with function calling enabled examines the input. This is what happens behind the scenes:

- Analyse Input: The model analyses the user’s intent based on the message's context.
- Check Functions: The system matches the intent with the available functions.
- Execute Function: The best-fitting function is executed, or the model responds without invoking a function if none is applicable.

To start the Swarm, instantiate a Swarm client:

In [4]:
client = Swarm()

And we call its run function and we enable debug mode to track interactions:

In [56]:
response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "Let me talk to a different agent"}],
    debug=True
)

[2024-11-21 12:57:24] Getting chat completion for...: [{'role': 'system', 'content': 'You are a helpful agent'}, {'role': 'user', 'content': 'Let me talk to a different agent'}]
[2024-11-21 12:57:25] Received completion: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_YzFP9WQZftROW10OSsqFqRUg', function=Function(arguments='{}', name='transfer_to_agent_b'), type='function')])
[2024-11-21 12:57:25] Processing tool call: transfer_to_agent_b with arguments {}
[2024-11-21 12:57:25] Getting chat completion for...: [{'role': 'system', 'content': "Always reply with 'Bzzzz'"}, {'role': 'user', 'content': 'Let me talk to a different agent'}, {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_YzFP9WQZftROW10OSsqFqRUg', 'function': {'arguments': '{}', 'name': 'transfer_to_agent_b'}, 'type': 'function'}], 'sender': 'Agent A'}, 

In [57]:
print(response.messages[-1].get('content'))

Bzzzz


## Context Variables

Context variables are a powerful feature in Swarm that allow agents to share additional information beyond the user's input. They function as a dictionary, allows you to pass key-value pairs that imrove the flow of communication and make the agent system more dynamic and personalised.

In simpler terms, they are a way to "pass notes" between agents For instance, if a customer interacts with a support agent, the agent might need to pass along the customer's name, issue, or preferences to another agent. Context variables ensure this information is available wherever needed.

To pass context variables, Swarm provides a Result class, which encapsulates the response, the target agent, and the context variables.

In [4]:
from swarm.types import Result

We create a function in the sales agent that passes context variables:

In [23]:
def talk_to_sales():
    print('Talk to sales function triggered')
    return Result(
        value="Done",
        agent=sales_agent,
        conext_variables={"department": "sales"}
    )

The sales agent will use the context variable customer_name in its response:

In [24]:
sales_agent = Agent(
    name = "Sales Agent",
    instructions=lambda context: f"Always greet the user {context['customer_name']}.",
    model = model,
)

We define the customer agent and include the talk_to_sales function in its list of callable functions:

In [25]:
customer_agent = Agent(
    name = "Customer Agent",
    model = model,
    functions = [talk_to_sales]
)

We use the Swarm run method to initialise the conversation:

In [26]:
customer_response = client.run(
    agent=customer_agent,
    messages=[{"role": "user", "content": "Transfer me to sales"}],
    context_variables={"customer_name": "Chris"},
    debug=True
)

[2024-11-20 13:17:19] Getting chat completion for...: [{'role': 'system', 'content': 'You are a helpful agent.'}, {'role': 'user', 'content': 'Transfer me to sales'}]
[2024-11-20 13:17:19] Received completion: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dY9CqsUKNus3QlZUAMQ3eZ6W', function=Function(arguments='{}', name='talk_to_sales'), type='function')])
[2024-11-20 13:17:19] Processing tool call: talk_to_sales with arguments {}
Talk to sales function triggered
[2024-11-20 13:17:19] Getting chat completion for...: [{'role': 'system', 'content': 'Always greet the user Chris.'}, {'role': 'user', 'content': 'Transfer me to sales'}, {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_dY9CqsUKNus3QlZUAMQ3eZ6W', 'function': {'arguments': '{}', 'name': 'talk_to_sales'}, 'type': 'function'}], 'sender': 'Customer Agent'}

In [27]:
print(customer_response.agent.name)
print(customer_response.context_variables)
print(customer_response.messages[-1].get("content"))

Sales Agent
{'customer_name': 'Chris'}
Hello Chris! I've connected you to sales. How can I assist you today?


## Case Study: Developing a Multi-Agent Customer Support System

Imagine you are building a customer support system for an online store that specialises in unique products like artisanal honey and beekeeping supplies. The goal is to create an intelligent system that can:

- Understand and route user queries to the appropriate support team.
- Handle common scenarios such as refunds and sales inquiries.
- Provide a personalised experience by dynamically responding to user input and utilising shared context.


The system will consist of three agents:

- Decision Agent: Acts as the first point of contact, determining the nature of the user’s query and routing it to the appropriate agent.
- Sales Agent: Handles inquiries about products, upsells items, and enthusiastically engages with customers.
- Refunds Agent: Manages refund requests.

Each agent is designed with specific instructions and functions to process user input, ensuring efficient query resolution.


In [59]:

# Define the function for processing refunds
def process_refund(context_variables):
    """Process a refund for an item."""
    item_id = context_variables.get('item_id', 'Unknown Item')
    reason = context_variables.get('reason', 'No reason provided')
    customer_name = context_variables.get('customer_name', 'Customer')
    print(f"Processing refund for {customer_name}: Item ID {item_id}, Reason: {reason}")
    return f"Success! Refund for {item_id} has been processed because {reason}."

# Define the Refunds Agent
refunds_agent = Agent(
    name="Refunds Agent",
    instructions="You handle refund requests. Use the 'process_refund' function to process refunds.",
    functions=[process_refund]
)

# Define the Sales Agent
sales_agent = Agent(
    name="Sales Agent",
    instructions="You provide information what customer asks regarding BeeBuzz products and assist with sales inquiries. BeeBuzz products are in unique products like artisanal honey and beekeeping supplies "
)

# Define the Decision Agent
def determine_agent(context_variables):
    """Determine which agent should handle the user's request."""
    query = context_variables.get('query', '').lower()
    if "refund" in query:
        return refunds_agent
    else:
        return sales_agent

decision_agent = Agent(
    name="Decision Agent",
    instructions="Based on the user's request, determine whether to transfer to the Sales Agent or Refunds Agent.",
    functions=[determine_agent]
)

# Function to run the conversation
def run_conversation(agent, query, context_variables):
    context_variables['query'] = query
    message = {"role": "user", "content": query}
    response = client.run(
        agent=agent,
        messages=[message],
        context_variables=context_variables,
        execute_tools=True,
        debug=False
    )
    print(response.agent.name)
    return response

# Full conversation simulation
def full_conversation():
    print("Welcome to BeeBuzz Support System. How can I help you today?")

    # Simulate a refund conversation
    refund_context = {
        "customer_name": "Chris",
        "item_name": "Bee Wax Candle",
        "item_id": "item_123",
        "reason": "Product didn't arrive"
    }
    refund_query = "I need a refund for my latest purchase."

    print("\n--- Refund Conversation ---")
    print(f"Customer Query: {refund_query}")
    response = run_conversation(decision_agent, refund_query, refund_context)
    print(response.messages[-1].get("content"))

    # Simulate a sales conversation
    sales_context = {
        "customer_name": "Chris",
        "item_name": "Honey Jar"
    }
    sales_query = "Tell me more about your latest product."

    print("\n--- Sales Conversation ---")
    print(f"Customer Query: {sales_query}")
    response = run_conversation(decision_agent, sales_query, sales_context)
    print(response.messages[-1].get("content"))

# Run the full conversation
full_conversation()


Welcome to BeeBuzz Support System. How can I help you today?

--- Refund Conversation ---
Customer Query: I need a refund for my latest purchase.
Processing refund for Chris: Item ID item_123, Reason: Product didn't arrive
Refunds Agent
Your refund has been successfully processed. If you have any further questions, feel free to ask!

--- Sales Conversation ---
Customer Query: Tell me more about your latest product.
Sales Agent
Our latest product at BeeBuzz is the "Alpine Wildflower Honey." This unique honey is made from the nectar of alpine wildflowers, which gives it a distinct, floral aroma and a delicate sweetness. It is harvested from hives situated in high-altitude areas, ensuring that it is not only pure but also rich in antioxidants and beneficial enzymes.

In addition to its delightful flavor, Alpine Wildflower Honey is perfect for those who appreciate artisanal, small-batch products. It can be enjoyed in various ways, such as drizzling over yogurt, spreading on toast, or sweet